In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import HashingVectorizer  
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk.data
from sklearn.metrics.pairwise import cosine_similarity
import time
import pandas as pd


data = pd.read_csv("/content/drive/MyDrive/SM Project/Posts.csv")
filteredData = data.dropna()
filteredData = filteredData.head(300000)

In [2]:
warnings.filterwarnings(action = 'ignore', category=DeprecationWarning)
postLinks = pd.read_csv("/content/drive/MyDrive/SM Project/PostLinks.csv")
postLinksDupl = postLinks[postLinks['LinkTypeId'] == 3]
len(postLinksDupl)

1096708

In [3]:
filteredData.head()

,Id,Body,Title,PostTypeId,Tags,CreationDate
0,337,<p>I am about to build a piece of a project th...,XML Processing in Python,1,<python><xml>,2008-08-02T03:35:55.697
1,469,<p>I am using the Photoshop's javascript API t...,How can I find the full path to a font from it...,1,<python><macos><fonts><photoshop>,2008-08-02T15:11:16.430
2,502,<p>I have a cross-platform (Python) applicatio...,Get a preview JPEG of a PDF on Windows?,1,<python><windows><image><pdf>,2008-08-02T17:01:58.500
3,535,<p>I am starting to work on a hobby project wi...,Continuous Integration System for a Python Cod...,1,<python><continuous-integration><extreme-progr...,2008-08-02T18:43:54.787
4,594,<p>There are several ways to iterate over a re...,cx_Oracle: How do I iterate over a result set?,1,<python><sql><database><oracle><cx-oracle>,2008-08-03T01:15:08.507


In [4]:
unansweredQues = pd.read_csv("/content/drive/MyDrive/SM Project/UnansweredPosts.csv")
print(len(unansweredQues))
unansweredQues['AcceptedAnswerId'].replace(-999, np.nan, inplace=True)
unansweredQues.dropna(subset=['AcceptedAnswerId'], inplace=True)
filUnAns = unansweredQues[unansweredQues['Tags'].str.contains("python")]

1000000


In [5]:
allIds = list(unansweredQues.Id)
titles = []
print(len(allIds))

unIds = []
for i in filteredData.index:
    if (filteredData['Id'][i] in allIds) and (len(unIds) <= 10):
        unIds.append(filteredData['Id'][i])
        titles.append(filteredData['Title'][i])
    else:
        break

print(unIds)

180080
[337, 469, 502, 535, 594, 683, 742, 766, 773, 972, 1171]


In [6]:
print(*titles, sep = '\n')


XML Processing in Python
How can I find the full path to a font from its display name on a Mac?
Get a preview JPEG of a PDF on Windows?
Continuous Integration System for a Python Codebase
cx_Oracle: How do I iterate over a result set?
Using 'in' to match an attribute of Python objects in an array
Class views in Django
Python and MySQL
How do I use itertools.groupby()?
Adding a Method to an Existing Object Instance
What is the most efficient graph data structure in Python?


In [7]:
import nltk
nltk.download('stopwords')
stopwords = stopwords.words('english')
st = WordNetLemmatizer()
def preprocessing(document):
    document_tokenized = [word.lower() for word in word_tokenize(document)]
    document_stopwords = [word for word in document_tokenized if word not in stopwords]
    english_punctuations = [":",".","\n","/","@", "\\","*","=","^",";","_","|",
                               '"',"' "," '","-",
                               "(",")",",",">","<",
                               "!","?","[","]","+",
                               "&","%","$","#","~","{","}"]
    document_filtered = [word for word in document_stopwords if not word in english_punctuations]
    document_stem = [st.lemmatize(word) for word in document_filtered]
    return document_stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
combinedText = []
combinedTextPreprocsd = []
count = 0
for i in filteredData.index:
    count = count+1
    text = str(filteredData['Title'][i]) + " " + str(filteredData['Body'][i])
    combinedText.append(text)
    combinedTextPreprocsd.append(preprocessing(text))
    if(count % 50000 == 0):
        print("Status of preprocessing text: " + str(count))
        
filteredData['tokenized_title'] = combinedTextPreprocsd
filteredData['combined_text'] = combinedText

filteredData.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Status of preprocessing text: 50000
Status of preprocessing text: 100000
Status of preprocessing text: 150000
Status of preprocessing text: 200000
Status of preprocessing text: 250000
Status of preprocessing text: 300000


,Id,Body,Title,PostTypeId,Tags,CreationDate,tokenized_title,combined_text
0,337,<p>I am about to build a piece of a project th...,XML Processing in Python,1,<python><xml>,2008-08-02T03:35:55.697,"[xml, processing, python, p, build, piece, pro...",XML Processing in Python <p>I am about to buil...
1,469,<p>I am using the Photoshop's javascript API t...,How can I find the full path to a font from it...,1,<python><macos><fonts><photoshop>,2008-08-02T15:11:16.430,"[find, full, path, font, display, name, mac, p...",How can I find the full path to a font from it...
2,502,<p>I have a cross-platform (Python) applicatio...,Get a preview JPEG of a PDF on Windows?,1,<python><windows><image><pdf>,2008-08-02T17:01:58.500,"[get, preview, jpeg, pdf, window, p, cross-pla...",Get a preview JPEG of a PDF on Windows? <p>I h...
3,535,<p>I am starting to work on a hobby project wi...,Continuous Integration System for a Python Cod...,1,<python><continuous-integration><extreme-progr...,2008-08-02T18:43:54.787,"[continuous, integration, system, python, code...",Continuous Integration System for a Python Cod...
4,594,<p>There are several ways to iterate over a re...,cx_Oracle: How do I iterate over a result set?,1,<python><sql><database><oracle><cx-oracle>,2008-08-03T01:15:08.507,"[cx_oracle, iterate, result, set, p, several, ...",cx_Oracle: How do I iterate over a result set?...


In [9]:
dictTitletoId = {}
for i in filteredData.index:
    dictTitletoId[filteredData['Title'][i]] = filteredData['Id'][i]
        
dictIdToTitle = {}
for i in filteredData.index:
    dictIdToTitle[filteredData['Id'][i]] = filteredData['Title'][i]

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

list_of_token_text = list(sent_to_words(filteredData.tokenized_title))

#tag each document 
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list_of_token_text)]
modelDoc2Vec = Doc2Vec(documents, vector_size=300, window=1, 
                min_count=3, workers=4, dm=0, alpha = 0.025,
                sample= 0.01, dm_concat = 1, dbow_words = 1)
modelDoc2Vec.save("model_doc2vec_python.model")

In [11]:
def itemDoc2Vec(id):
    try:
        title = filteredData.loc[filteredData['Id'] == id]['Title'].tolist()[0]
    except:
        title = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
    return title

def recommendDoc2Vec(id):
    print("Recommending posts similar to: " + itemDoc2Vec(id)) 
    print("----------------------------------------------------------")
    title_input = ''
    body_input = ''
    for i in range(0, len(filteredData)):
        if filteredData['Id'][i] == id:
            title_input = filteredData['Title'][i]
            body_input = filteredData['Body'][i]
            break
    if title_input == '' or body_input == '':
        title_input = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
        body_input = unansweredQues.loc[unansweredQues['Id'] == id]['Body'].tolist()[0]
    test_input = title_input + " " + body_input
    test_input = preprocessing(test_input)
    test_vector = modelDoc2Vec.infer_vector(test_input)
    similar_cases = modelDoc2Vec.docvecs.most_similar(positive = [test_vector], topn=11)
    top_10_similar_cases = similar_cases [1:11]
    print("You may like to read: \n")
    for i in range(0, len(top_10_similar_cases)):
        print("PostId: "+ str(filteredData.loc[top_10_similar_cases[i][0]]['Id']) +
          ". "+ filteredData.loc[top_10_similar_cases[i][0]]['Title'] + " (score:" + str(round(top_10_similar_cases[i][1], 3)) + ")")
    return top_10_similar_cases[0][1]


In [12]:
simScoreListDoc2Vec = []
for uniqid in unIds:
    simScore = recommendDoc2Vec(uniqid) #Passing the postId of an unanswered post
    simScoreListDoc2Vec.append("Highest score for Post Id " + str(uniqid) + " is = " + str(simScore))
print(*simScoreListDoc2Vec, sep = "\n")

Recommending posts similar to: XML Processing in Python
----------------------------------------------------------
You may like to read: 

PostId: 301493. Which language is easiest and fastest to work with XML content? (score:0.66)
PostId: 3325028. What are the technologies that i should use for developing SaaS. Software as a service (score:0.658)
PostId: 331600. How to write a Web Service for Google App Engine? (score:0.658)
PostId: 159802. Python Web Services (score:0.647)
PostId: 804992. How to consume XML from RESTful web services using Django / Python? (score:0.646)
PostId: 20718662. How to create a financial evaluation model using python? (score:0.642)
PostId: 4641187. PHP or Python for Image Processing? (score:0.641)
PostId: 2714978. Creating WCF Services using Dynamic Languages and DLR (score:0.638)
PostId: 359201. How do I create a Web Service in IronPython? (score:0.637)
PostId: 2085430. populating data from xml file to a sqlite database using python (score:0.637)
Recommendin

In [13]:
import nltk
nltk.download('averaged_perceptron_tagger')
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
def preprocess_tf_idf(text):
    text = text.lower()
    temp_sent =[]
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES: 
            lemmatized = st.lemmatize(word, 'v')
        else:
            lemmatized = st.lemmatize(word)
        if lemmatized not in stopwords and lemmatized.isalpha():
            temp_sent.append(lemmatized)
          
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    return finalsent
  
filteredData["combined_text_processed"]= filteredData['combined_text'].apply(preprocess_tf_idf)

filteredData.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


,Id,Body,Title,PostTypeId,Tags,CreationDate,tokenized_title,combined_text,combined_text_processed
0,337,<p>I am about to build a piece of a project th...,XML Processing in Python,1,<python><xml>,2008-08-02T03:35:55.697,"[xml, processing, python, p, build, piece, pro...",XML Processing in Python <p>I am about to buil...,xml processing python p build piece project ne...
1,469,<p>I am using the Photoshop's javascript API t...,How can I find the full path to a font from it...,1,<python><macos><fonts><photoshop>,2008-08-02T15:11:16.430,"[find, full, path, font, display, name, mac, p...",How can I find the full path to a font from it...,find full path font display name mac p use pho...
2,502,<p>I have a cross-platform (Python) applicatio...,Get a preview JPEG of a PDF on Windows?,1,<python><windows><image><pdf>,2008-08-02T17:01:58.500,"[get, preview, jpeg, pdf, window, p, cross-pla...",Get a preview JPEG of a PDF on Windows? <p>I h...,get preview jpeg pdf window p python applicati...
3,535,<p>I am starting to work on a hobby project wi...,Continuous Integration System for a Python Cod...,1,<python><continuous-integration><extreme-progr...,2008-08-02T18:43:54.787,"[continuous, integration, system, python, code...",Continuous Integration System for a Python Cod...,continuous integration system python codebase ...
4,594,<p>There are several ways to iterate over a re...,cx_Oracle: How do I iterate over a result set?,1,<python><sql><database><oracle><cx-oracle>,2008-08-03T01:15:08.507,"[cx_oracle, iterate, result, set, p, several, ...",cx_Oracle: How do I iterate over a result set?...,iterate result set p several way iterate resul...


In [ ]:
from sklearn.metrics.pairwise import linear_kernel
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=50, stop_words='english')
filteredData_new = filteredData.head(10000)

tfidf_matrix = tf.fit_transform(filteredData_new['combined_text_processed'])
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
results = {} # dictionary created to store the result in a dictionary format (ID : (Score,item_id))#

for idx, row in filteredData_new.iterrows(): 
    # the below code 'similar_indice' stores similar ids based on cosine similarity. sorts them in ascending order. [:-10:-1] is then used so that the indices with most similarity are got. 0 means no similarity and 1 means perfect similarity#
    similar_indices = cosine_similarities[idx].argsort()[:-10:-1] 

    #stores 10 most similar books, you can change it as per your needs
    similar_items = [(cosine_similarities[idx][i], filteredData_new['Id'][i]) for i in similar_indices]
    results[row['Id']] = similar_items[1:]

In [ ]:
def itemTfIdf(id):
    try:
        title = filteredData_new.loc[filteredData_new['Id'] == id]['Title'].tolist()[0]
    except:
        title = unansweredQues.loc[unansweredQues['Id'] == id]['Title'].tolist()[0]
    return title

def recommendTfIdf(id):
    print("Recommending posts similar to: " + itemTfIdf(id)) 
    print("----------------------------------------------------------")
    recs = results[id][:11]
    print("You may like to read: \n")
    for rec in recs:
        print("Post Id:" + str(dictTitletoId[itemTfIdf(rec[1])]) + ". " + itemTfIdf(rec[1]) + " (score:" + str(round(rec[0], 3)) + ")")
    return recs[0][0]


In [ ]:
simScoreListTfIdf = []
for uniqid in unIds:
    simScore = recommendTfIdf(uniqid) #Passing the postId of an unanswered post
    simScoreListTfIdf.append("Highest score for Post Id " + str(uniqid) + " is = " + str(simScore))
print(*simScoreListTfIdf, sep = "\n")